# Interpreter

## Imports

In [9]:
from ipynb.fs.defs.environment import *
import re
from collections import namedtuple
import os
import glob

## Tokenizing Code

### Exceptions

In [10]:
class CodeException(Exception):
    def __init__(self, message, position):
        super().__init__(message)
        self.position = position

### Token Definitions

In [3]:
class TokenDef(namedtuple('TokenDef', ('name', 'pattern', 'value_filter'))):
    def __repr__(self):
        return 'TokenType.' + self.name
    
Token = namedtuple('Token', ('type', 'value', 'slice'))
    
class TokenType(object):
    _defs = [
        TokenDef('left_paren', '(', None),
        TokenDef('right_paren', ')', None),
        TokenDef('whitespace', re.compile('\s+'), None),
        TokenDef('integer', re.compile('[0-9]+'), int),
        TokenDef('move', 'move', None),
        TokenDef('turn', 'turn', None),
        TokenDef('left', 'lt', None),
        TokenDef('right', 'rt', None),
        TokenDef('repeat', 'repeat', None)
    ]
    
for def_ in TokenType._defs:
    setattr(TokenType, def_.name, def_)

### First Token In Code Extractor

In [4]:
def first_token(code, start=0):
    code = code[start:]
    token = None
    for type_ in TokenType._defs:
        name, pattern, value_filter = type_
        
        if isinstance(pattern, str):
            if not code.startswith(pattern):
                continue
            match_value = pattern
        else:
            match = pattern.match(code)
            if not match:
                continue
            match_value = match.group(0)
        
        len_ = len(match_value)
        
        if value_filter is not None:
            match_value = value_filter(match_value)
        
        token = Token(type_, match_value, slice(start, start + len_))
    return token

### Code Lexer

In [5]:
def lex(code):
    start = 0
    while start < len(code):
        token = first_token(code, start)
        if token == None:
            raise CodeException("Unexpected code at position ", start)
        start = token.slice.stop
        if token.type is TokenType.whitespace:
            continue
        yield token

## Code Executor

### Tokens To Executable Segment Parser

In [6]:
class Parser:
    
    def _parse_move(tokens, repeat_stack, exec_queue):
        steps_ = next(tokens)
        if steps_.type is not TokenType.integer:
               raise CodeException('Expected number at position ', steps_.slice.start)
        if len(repeat_stack) == 0:
            exec_queue.append(MoveSegment(steps_.value))
        else:
            repeat_segment = repeat_stack.pop()
            repeat_segment.body.append(MoveSegment(steps_.value))
            repeat_stack.append(repeat_segment)
            
    def _parse_turn(tokens, repeat_stack, exec_queue):
        dir_ = next(tokens)
        angle_ = next(tokens)
        if dir_.type not in [TokenType.left, TokenType.right]:
            raise CodeException('Expected direction at position ', dir_.slice.start)
        if angle_.type is not TokenType.integer:
            raise CodeException('Expected number at position ', angle_.slice.start)
        if len(repeat_stack) == 0:
            exec_queue.append(TurnSegment(dir_.value,angle_.value))
        else:
            repeat_segment = repeat_stack.pop()
            repeat_segment.body.append(TurnSegment(dir_.value,angle_.value))
            repeat_stack.append(repeat_segment)
    
    def _start_repeat(tokens, repeat_stack, exec_queue):
        n_ = next(tokens)
        if n_.type is not TokenType.integer:
            raise CodeException('Expected number at position ', n_.slice.start)
        start_ = next(tokens)
        if start_.type is not TokenType.left_paren:
            raise CodeException('Expected opening paren at position ', start_.slice.start)
        repeat_stack.append(RepeatSegment(n_.value, []))
        
    def _end_repeat(tokens, repeat_stack, exec_queue):
        if len(repeat_stack) == 0:
            raise CodeException('Unexpected closing paren.', None)
        if len(repeat_stack) == 1:
            exec_queue.append(repeat_stack.pop())
        else:
            inner_repeat_stack = repeat_stack.pop()
            outer_repeat_stack = repeat_stack.pop()
            outer_repeat_stack.body.append(inner_repeat_stack)
            repeat_stack.append(outer_repeat_stack)

### Parse And Execute Code

In [7]:
def execute(code, label, to_save = False):
    try:
        tokens = lex(code)
        repeat_stack = []
        exec_queue = []
        start()

        while True:
            token = next(tokens)
            if token.type is TokenType.move:
                Parser._parse_move(tokens, repeat_stack, exec_queue)        
            elif token.type is TokenType.turn:  
                Parser._parse_turn(tokens, repeat_stack, exec_queue)
            elif token.type is TokenType.repeat:
                Parser._start_repeat(tokens, repeat_stack, exec_queue)
            elif token.type is TokenType.right_paren:                     
                Parser._end_repeat(tokens, repeat_stack, exec_queue)

    except CodeException as e:
        print('Code Error: ' + str(e) + str(e.position))

    except StopIteration:   
        while len(exec_queue) > 0:
            segment = exec_queue.pop(0)
            segment._exec()
        if to_save:
            save_graphic(label)
        
    except Exception as e:
        print('Error: ' + str(e))

    finally:
        end()

### Loading Code To Execute

In [8]:
base_path = "E:/Alphabets Project"
os.chdir(base_path)

In [19]:
alphabets_folders = glob.glob('Alphabet*')

for folder in alphabets_folders:
    alphabet_path = base_path + '/' + folder
    os.chdir(alphabet_path)
    os.system('mkdir Images')
    os.chdir('Codes')
    
    char_folders = glob.glob('*')
    
    for char_folder in char_folders:
        images_folder_path = alphabet_path + '/Images'
        os.chdir(images_folder_path)
        os.system('mkdir ' + char_folder)
        char_folder_path = base_path + '/' + folder + '/Codes/' + char_folder
        os.chdir(char_folder_path)
        
        instances = glob.glob('*')
        
        for instance in instances:
            os.chdir(char_folder_path)
            file = open(instance, 'r')
            code = file.read()
            file.close()
            os.chdir(images_folder_path + '/' + char_folder)
            label = os.path.splitext(instance)[0]
            execute(code, label, True)
            
    print(folder + ' DONE')

Alphabet 0l 7c DONE
